In [ ]:
!pip install -q gradio
# Install Pytorch library
!pip install segmentation-models-pytorch

In [ ]:
import gradio as gr
from google.colab import drive
import os, cv2, json
from os import getcwd
import numpy as np
import typing as T
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import matplotlib.image as mpimg
import math
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp


In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder path
folder_path = "/content/drive/MyDrive/Your_Folder_Name"

# Change the working directory to the specified path
os.chdir(folder_path)

Mounted at /content/drive


In [ ]:
dataset_path = os.path.join(os.getcwd(), 'dataset')

model = smp.Unet(
            encoder_name    = "efficientnet-b0",
            encoder_weights = None,
            in_channels     = 1,
            classes         = 5,
        )
ckpt_path = os.path.join(folder_path,'training/dump/trained_100Epoch/training_checkpoint.pth')
ckpt = torch.load(ckpt_path,map_location=torch.device('cpu'))
model.load_state_dict(ckpt['state_dict'])
model.eval()
model.to('cpu')

In [ ]:
def process_image(pil_image):
    # Convert PIL image to NumPy array
    numpy_array = np.array(pil_image)
    # Check the number of channels
    num_channels = numpy_array.shape[2] if len(numpy_array.shape) == 3 else 1

    # Convert the image to grayscale if it's not already
    if num_channels > 1:
        gray_image = cv2.cvtColor(numpy_array, cv2.COLOR_RGB2GRAY)
    else:
        gray_image = numpy_array
    img = gray_image

    IMGSZ = 512
    class_names = ["FP", "FPD", "ASH", "bf"]
    # Augment an image
    tensor_image = A.Compose(
        [
            A.Resize(IMGSZ, IMGSZ, interpolation=cv2.INTER_NEAREST, always_apply=False, p=1),
            ToTensorV2(),
        ],
    )(image=img)['image']

    with torch.no_grad():
        preds = torch.softmax(model(tensor_image.to('cpu').unsqueeze(0).float()).squeeze(0), dim=0)
    preds = (preds > 0.5).cpu().numpy()

    def get_class_colors(classes):
        colors = {
            'FP': [128, 0, 128],  # Purple for class 1
            "FPD": [0, 255, 255],  # Cyan for class 2
            "ASH": [0, 128, 0],  # Green for class 3
            "bf": [255, 0, 0],  # Red for class 4
        }
        return colors

    colors = get_class_colors(class_names)

    rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    rgb = A.Compose(
        [
            A.Resize(IMGSZ, IMGSZ, interpolation=cv2.INTER_NEAREST, always_apply=True, p=1),
        ],
    )(image=rgb)['image']

    for ch in range(3):
        for idx, cls_name in enumerate(class_names):
            rgb[:, :, ch][preds[idx + 1] != 0] = colors[cls_name][ch]

    # Draw arrow from the center of FP towards FPD if FPD is present
    if np.max(preds[2]) != 0:
        cords_FP = np.where(preds[1] == 1)
        cy_fp, cx_fp = int(np.mean(cords_FP[0])), int(np.mean(cords_FP[1]))

        cords_FPD = np.where(preds[2] == 1)
        cy_fpd, cx_fpd = int(np.mean(cords_FPD[0])), int(np.mean(cords_FPD[1]))

        # Calculate the vector from the center of FP to the center of FPD
        vector_fp_fpd = np.array([cx_fpd - cx_fp, cy_fpd - cy_fp])

        # Scale the vector to make the arrow longer (e.g., multiplying by 2)
        scaled_vector = 1.1 * vector_fp_fpd

        # Calculate the endpoint of the arrow after scaling
        endpoint = (cx_fp + scaled_vector[0], cy_fp + scaled_vector[1])

        # Draw a small circle at the beginning of the arrow
        cv2.circle(rgb, (cx_fp, cy_fp), radius=10, color=[0, 0, 255])  # Blue color for the circle

        # Draw the arrow
        arrow_color = [0, 0, 255]  # Blue color for the arrow
        rgb = cv2.arrowedLine(rgb, (cx_fp, cy_fp), (int(endpoint[0]), int(endpoint[1])), arrow_color, thickness=2)

        # Convert BGR image to RGB
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        # Convert NumPy array to PIL Image
        out_img = Image.fromarray(rgb)

    return out_img


In [ ]:
# Create a Gradio Interface
demo = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="pil"),
    examples=[
        os.path.join(dataset_path, 'test/images', 'Fadul 5-1 3DVM.png'),
        os.path.join(dataset_path, 'test/images', 'Fadul 5-2 3DVM.png'),
        os.path.join(dataset_path, 'test/images', 'Fadul 6-1 3DVM.png'),
        os.path.join(dataset_path, 'test/images', 'Fadul S 3DVM.png'),
    ],
    live=True,
)

# Launch the Gradio Interface
if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7f4a10c5ec920fa79e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
